<a href="https://colab.research.google.com/github/PerceptronV/Miscellaneous/blob/master/Deep_Learning_Crash_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DLCC-10
####A 10 minute crash course in Deep Learning

/_By Vincent Song_

---


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PerceptronV/Miscellaneous/blob/master/Deep_Learning_Crash_Course.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PerceptronV/Miscellaneous/blob/master/Deep_Learning_Crash_Course.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

##Introduction
```
Artificial intelligence has taken the world by storm. But how exacty does it work, and what lies ahead?
```

We'll be exploring all these and some more. By the end of this quick, interactive notebook, you'll have have trained your own text-generating AI!

Let's take a glimpse of greatness:
![A blend of human and machine-generated Haikus](https://github.com/PerceptronV/Miscellaneous/raw/master/haiku_guess.PNG)

##A Map of AI
![3 Stages of AI](https://image.slidesharecdn.com/futureofai20190507v7-190507093643/95/future-of-ai-20190507-v7-5-638.jpg?cb=1557221878)

_After Dario Gil, IBM_

| Narrow AI | Broad AI | General AI |
| ------------- | ------------- | ------------- |
| Single-task, single-domain | Multi-task, multi-domain | Cross-domain learning and reasoning |
| Superhuman performace on specific tasks | Explainable | Broad autonomy, superhuman performace in every area |

\

\

![The map of AI](https://top6sites.com/wp-content/uploads/2020/05/AI-vs-ML-vs-Deep-Learning.png)

##A bit on theory

###Let's get our hands dirty!
_Time for the code to come in!_

For this simple demonstration, we will be coding in Python - the leading language for data scientists.

__SPOILER ALERT__: Implementing a basic neural network and training it from scratch is going to take an absurdly long time, and is well beyond what we can do in 10 minutes. Therefore, to speed up the process, we will be using _TensorFlow_, a popular machine learning library, and _Keras_, a framework built on top of it.

###Importing Libraries

In [ ]:
#Let us first import the libraries we need
import tensorflow as tf
from tensorflow import keras

import time, os, math
import numpy as np
from tqdm import tqdm

###Creating a simple GRU layer

##Creating our AI system

###Downloading the data
We are going to teach an AI how to generate realistic-looking text of a certain type based on data in a _.txt_ file. Here are few prepared styles for you to choose from: 


| Text Type  | Description | Adress / URL |
| ------------- | ------------- | ------------- |
| Haikus | A form of 3-line poetry, originating from Japan. | [https://raw.githubusercontent.com/PerceptronV/Apollo-Psi/master/Haikus.txt](https://raw.githubusercontent.com/PerceptronV/Apollo-Psi/master/Haikus.txt) |
| Shakespeare | _Everyone knows him_ | [https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt](https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt) |
| Emily Dickinson | American poet. Her works were unconventional and often centred around themes like death or religion. | [https://raw.githubusercontent.com/PerceptronV/Miscellaneous/master/Emily_Dickinson.txt](https://raw.githubusercontent.com/PerceptronV/Miscellaneous/master/Emily_Dickinson.txt) |
| Virgil | __Latin__ full text from the Roman epic poem _The Aeneid_ by Virgil | [https://raw.githubusercontent.com/PerceptronV/Miscellaneous/master/aeneid-lat.txt](https://raw.githubusercontent.com/PerceptronV/Miscellaneous/master/aeneid-lat.txt) |



Choose the type you'd like, copy its link address, then run the next block and paste in the url.

_(To customise, you can also input an url of any text file you like, as long as it is encoded in utf-8 and is public.)_

In [ ]:
URL = input("Enter the URL of the text file you'd like to use:\n")

if os.path.exists('src.txt'):
  os.remove('src.txt')
fp = keras.utils.get_file('src.txt', URL, cache_subdir = '/content')

###Read and tokenize the text
Before we do anything, we have to load the text in the file you've downloaded. Then we convert all the characters in that text into integers, called _tokens_. The whole process is called _tokenization_.

In [ ]:
text = open(fp,'rb').read().decode(encoding='utf-8')

vocab = sorted(set(text))
TOKEN_SIZE = len(vocab)

get_tok = {u:i for i, u in enumerate(vocab)}
get_char = np.array(vocab)

In [ ]:
# Let's visualize the mapping between the first set of tokens and characters
# \n is the newline character

print('token\t----->\tcorresponding character\n')
for i in range(0,7):
  print('{}\t----->\t{}'.format(i, get_char[i].replace('\n', '\\n')))

In [ ]:
# Defining helper functions to make tokenizing and decoding easier

def tokenize(s):
  return [get_tok[i] for i in s]

def decode(l):
  ret=''
  for i in l:
    ret+=get_char[i]
  return ret

###Creating a dataset
The training process is really simple. We separate the text into sequences of tokens, all of the same length. We then give the model an input token, and train it to predict the most probable next token.

This is like the next-word prediction function on your smartphones, just specialized in the text you're giving it.

In [ ]:
# Converting the whole text into tokens and turning it into tf.data.Dataset

token_text = tokenize(text)
token_data = tf.data.Dataset.from_tensor_slices(token_text)

In [ ]:
# Breaking the text into sequences

SEQ_LEN = 100   # This is the sequence length. You can tweak this value.

seq_data = token_data.batch(SEQ_LEN, drop_remainder = True)

print('Example sequence:\n')
for i in seq_data.take(1):
  print(decode(i).replace('\n', '\\n'))

In [ ]:
BATCH_SZ = 64  # The batch-size parameter. It's usually between 64-256.

# Creating input-target pairs

def to_inp_targ(seq):
  '''
    Function for turning an individual sequence into input-target pairs
  '''
  return seq[:-1], seq[1:]

# Applying the to_inp_targ function to all sequences, then batching the dataset
dataset = seq_data.map(to_inp_targ)
batch_dataset = dataset.batch(BATCH_SZ, drop_remainder = True)

In [ ]:
# visualizing input-target pairs

for inp_seq, out_seq in dataset.take(1):
  print('Input')
  print(decode(inp_seq).replace('\n', '\\n'))
  print('\nOutput')
  print(decode(out_seq).replace('\n', '\\n'))

###Making the model
We're going make our own Model class with keras.Model.

The model takens in a token, turns it into an embedding vector, then passes it through a Recurrent Neural Network, a GRU in this case. It then passes through a fully connected neural network, the size of which equals the number of tokens (i.e. the number of different characters we have in our text).

This is because we want the model to output a probability distribution on all the possible next tokens, hence achieving next-word prediction.

Here is a visual diagram of how the model will operate:
![Diagram of next-word-prediction framework.](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

_Image Copyright 2019 The TensorFlow Authors._

In [ ]:
# Model parameters

EMB_DIM = 1024  # This is the size of the embedding vector
UNITS = 1024  # This is the number of RNN units in the model

'''
0 <= dropout < 1

Dropout controlles how much the model forgets certain information 
during training.

I.e.  A dropout of 0 means that the model remembers everything its 
      taught at evert moment in the training.

Dropout is used to combat overfitting, which happens when your model
learns the data so well it begins to memorize it. 

So, if you find that after training below, the model memorises the
dataset its given instead of generating anything new, try increasing
the dropout value.
'''
DROPOUT = 0.0

In [ ]:
class Model(keras.Model):
  def __init__(self, embedding_dim, units, token_size, dropout):
    super(Model,self).__init__(name = 'Baseline')
    
    # Defining the layers we need
    self.embedding = keras.layers.Embedding(token_size, embedding_dim)
    self.gru1 = keras.layers.GRU(units, recurrent_initializer = 'glorot_uniform',
                                 return_sequences = True, dropout = dropout)
    self.out = keras.layers.Dense(token_size, activation = keras.activations.softmax)

  def call(self, input):
    # Passing the input forward through our layers

    x = self.embedding(input)
    x = self.gru1(x)
    return self.out(x)

In [ ]:
model = Model(EMB_DIM, UNITS, TOKEN_SIZE, DROPOUT)

###Defining the loss function and optimizer
Since the task is inherently about classifying the most probable next token, we use the cross-entropy loss.

Cross-entropy loss formula:

$-\sum\limits_{i=1}^M y_i\log(p_i)$

$y_i$ is the label of the $i^{th}$ token. In classification, this value is either 0 or 1, where 1 means that the token is the correct prediction, and 0 states otherwise.

$p_i$ is the probability assigned by the model on the $i^{th}$ token.

$M$ is the token size.


---



As for the optimizer, we will use the Adam Optimizing Algorithm to help train our model weights. More information of Adam is [available here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/#:~:text=Adam%20is%20an%20optimization%20algorithm,iterative%20based%20in%20training%20data.&text=The%20algorithm%20is%20called%20Adam.).

In [ ]:
loss_function = keras.losses.sparse_categorical_crossentropy
optimizer = tf.keras.optimizers.Adam()

# Remark: There's nothing magical about the name 
# `sparse_categorical_crossentropy`. It's just some technical
# details. The loss function is still calculating the 
# cross-entropy loss by the formula given above.

###Training our network
We are going to implement the training loop and gradient descent here.

In [ ]:
def train_step(input, labels):
  
  with tf.GradientTape() as tape:
    predictions = model(input)
    loss = loss_function(labels, predictions)
  
  # calculate gradients with respect to model variables
  gradients = tape.gradient(loss, model.trainable_variables)

  # update gradients with optimzer, based on the gradients
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return tf.reduce_mean(loss)

We're just iterating over the data in the code below. Nothing mysterious.

In [ ]:
def train(epochs, steps_per_epoch):

  for i in range(epochs):
    total_loss = []
    start = time.time()

    for step, batch in enumerate(batch_dataset.repeat()):
      # Due to the way the dataset is structured, each batch includes
      # two components: batch[0] is the inputs, batch[1] is the labels

      step_loss = train_step(batch[0], batch[1])
      total_loss.append(step_loss)

      if step % 30 == 0:
        print('Step: {}\tElapsed time: {}s\tLoss: {}'.format(
            step, time.time()-start, np.mean(np.asarray(total_loss))
        ))
      
      if (step+1) == steps_per_epoch:
        # Break out of training loop once the set number of gradient
        # descent steps is reached
        break
    
    print('> Epoch: {}\tTime: {}s\tLoss: {}\n'.format(
        i+1, time.time()-start, np.mean(np.asarray(total_loss))
    ))

In [ ]:
'''
Epochs is the number of rounds we train an AI. Generally, the more you
train, the better. But there is a point at which the model starts to
overfit, or in other words, starts to model the data so well it memorises
it. 

If overfitting happens to you, try reducing the number of epochs or tweak
the dropout parameter above.

If the AI isn't generating meaningul stuff after you've trained it, try
increasing the number of epochs, or change the model structure altogether.
'''

EPOCHS = 30
STEPS = 128  # The number of gradient descent steps to perform in each epoch

###_Actually_ running the __`train`__ function
Behold - the FUN!!!

But, training does take a while. Be patient. Watch the loss drop - it's most satisfying.

In [ ]:
train(EPOCHS, STEPS)

##Generating text!!
Let's get started with our system!

---

The following code inputs a prompt to the model, and then samples a token from the probability distribution generated by our model. 

The distribution is divided by the temperature. A higher temperature means there is a higher chance that some tokens with a lower assigned popability might be sampled, often making the generated text surprising. A temperature of 0 is basically the same as selecting the token with highest probability.

Temperature is always a non-negative integer. Tweak around to generate balance the creativity with the uniformity of the text.

Once a token is sampled, it is added to the original prompt, and this continued prompt is then fed back into the model, which then generates a probability distribution for us to sample from, and so we have another token, and so on etc.

This generation technique continues until the model has generated the desired number of outputs, the `generation_length`.

In [ ]:
def generate_text(temperature, generation_length, prompt):
    
    # Turning input text into tokens
    input = tf.expand_dims(tokenize(prompt), 0)

    output = []  # Initializing a list to store generated tokens
    model.reset_states()  # Resetting the idden states of our RNN model

    for i in range(generation_length):
      
      preds = model(input)  # Making a next-word prediction with our model
      preds = tf.squeeze(preds, 0)
      preds = preds / temperature  # Dividing distribution by temperature

      # Sampling from the distribution
      id = tf.random.categorical(preds, num_samples=1)[-1,0].numpy()

      input = tf.expand_dims([id],0)
      output.append(id)
      
    return decode(output)

In [ ]:
# Defining generation parameters

TEMPERATURE = 0.4
GENERATION_LENGTH = 100

In [ ]:
prompt = '\n\nfireflies\n'
generate_text(TEMPERATURE, GENERATION_LENGTH, prompt)